In [ ]:
%matplotlib inline

In [ ]:
%%capture
# uncomment to install spatial pandas once
!pip install spatialpandas

In [ ]:
%%time
import pandas as pd
import geopandas as gpd
huc10 = gpd.read_parquet('huc10.parq')

In [ ]:
%%time
huc10.plot("bias")

In [ ]:
%%time
import spatialpandas as sp
huc10 = sp.GeoDataFrame(huc10)

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
cluster

In [ ]:
%%time
import dask.dataframe as dd
ddf = dd.from_pandas(huc10, npartitions=8).pack_partitions(npartitions=8).persist()

In [ ]:
import datashader as ds
import holoviews as hv

hv.extension("bokeh")

In [ ]:
def callback(x_range, y_range):
    cvs = ds.Canvas(plot_width=650, plot_height=400, x_range=x_range, y_range=y_range)
    agg = cvs.polygons(ddf, geometry='geometry', agg=ds.mean('bias'))
    return hv.Image(agg).opts(cmap="viridis", tools=['hover']).opts(width=650, height=400)

In [ ]:
hv.DynamicMap(
    callback, streams=[hv.streams.RangeXY()]
)

# Thinking about hvplot (not working)

In [ ]:
# import numpy as np
# import hvplot.pandas
# import hvplot.xarray
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
# import cartopy.crs as ccrs

In [ ]:
# p = huc10.hvplot.polygons(c='bias')

In [ ]:
# g = rasterize(p, aggregator='mean', x_sampling=0.02, y_sampling=0.02, width=500).opts(tools=['hover'],
#                 aspect='equal', logz=True, cmap='viridis', clim=(1e-2, np.nan))

In [ ]:
# g * gv.tile_sources.OSM